# Trích Xuất và Cấu Trúc Dữ Liệu SMS
File csv được xuất bằng chức năng Export sau khi đã thực hiện Backup qua ứng dụng SMS Backup & Restore. 

Link Google Play Store: https://play.google.com/store/apps/details?id=com.riteshsahu.SMSBackupRestore&hl=vi&pli=1.

Notebook này thực hiện các bước sau:
1. Đọc dữ liệu SMS từ file CSV gốc
2. Phân tích và xử lý dữ liệu khuyết
3. Lọc chỉ lấy tin nhắn nhận được (Type = 'Received')
4. Tạo dữ liệu có cấu trúc với các đặc trưng:
   - `content`: Nội dung tin nhắn
   - `label`: Nhãn phân loại spam/ham (cần gán thủ công)
   - `has_url`: Có chứa URL hay không
   - `has_phone_number`: Có chứa số điện thoại hay không
   - `sender_type`: Loại người gửi (brandname/shortcode/personal_number)

## Import thư viện cần thiết

In [44]:
import pandas as pd
import re
import os


## Đọc và Khảo Sát Dữ Liệu Gốc

Đọc các file CSV chứa dữ liệu SMS đã xuất từ điện thoại và kiểm tra thông tin cơ bản về dataset.

In [45]:
# Đọc tất cả các file csv trong thư mục raw_data và gộp dữ liệu

raw_data_dir = "../raw_data"
all_files = [f for f in os.listdir(raw_data_dir) if f.endswith('.csv')]
df_list = []

for f in all_files:
    file_path = os.path.join(raw_data_dir, f)
    # Đọc file với pandas, xử lý header linh hoạt/thích hợp
    try:
        tmp = pd.read_csv(file_path, encoding='utf-8')
    except UnicodeDecodeError:
        tmp = pd.read_csv(file_path, encoding='utf-8-sig')
    df_list.append(tmp)

# Gộp các dataframe lại
original_data = pd.concat(df_list, ignore_index=True)

original_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2841 entries, 0 to 2840
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Type           2841 non-null   object
 1   Date           2841 non-null   object
 2   Name / Number  2841 non-null   object
 3   Sender         2613 non-null   object
 4   Content        2841 non-null   object
dtypes: object(5)
memory usage: 111.1+ KB


## Kiểm tra trùng lặp nội dung tin nhắn và loại bỏ

Ở đây ta sẽ tiến hành kiểm tra xem có việc trùng lặp NỘI DUNG các tin nhắn hay không, tức là kiểm tra cột "Content". Nếu có thì tiến hành loại bỏ.

### Kiểm tra trùng lặp

In [46]:
# Kiểm tra số lượng tin nhắn trùng lặp theo cột "Content"
num_total = original_data.shape[0] 
num_content_unique = original_data['Content'].nunique()
num_content_duplicates = num_total - num_content_unique

print(f"Tổng số tin nhắn: {num_total}")
print(f"Số lượng nội dung tin nhắn duy nhất: {num_content_unique}")
print(f"Số lượng nội dung tin nhắn bị trùng lặp: {num_content_duplicates}")

# Các nội dung bị trùng lặp (nội dung xuất hiện nhiều hơn 1 lần)
duplicate_contents = original_data['Content'].value_counts()
duplicate_contents = duplicate_contents[duplicate_contents > 1]
if not duplicate_contents.empty:
    print("\nMột số nội dung tin nhắn bị trùng lặp:")
    print(duplicate_contents)
else:
    print("Không có nội dung tin nhắn nào bị trùng lặp.")


Tổng số tin nhắn: 2841
Số lượng nội dung tin nhắn duy nhất: 1993
Số lượng nội dung tin nhắn bị trùng lặp: 848

Một số nội dung tin nhắn bị trùng lặp:
Content
Quy khach da dang ky thanh cong goi ST5KM. Phi 5.000d/ngay co 1GB su dung den 24h, het luu luong truy cap theo goi Mobile Internet dang su dung (neu co). DV Khong gia han theo ngay. Soan KTTK de kiem tra luu luong tin nhan gui 191 hoac truy cap https://myvt.page.link/kttk1. LH 198 (0d). Tran trong.                                                                                                              30
Viettel TB: Goi ST5KM da het thoi gian su dung. De tiep tuc su dung goi ST5KM, Quy khach vui long dang ky tai https://myvt.page.link/ST5KM. Chi tiet LH 198 (0d). Tran trong                                                                                                                                                                                                                                             29
D6V               

### Loại bỏ trùng lặp

In [47]:
# Loại bỏ tin nhắn trùng lặp, chỉ giữ lại 1 lần duy nhất cho mỗi nội dung "Content"
original_data = original_data.drop_duplicates(subset=['Content'], keep='first').reset_index(drop=True)

# Kiểm tra lần 2
num_total = original_data.shape[0]
num_content_unique = original_data['Content'].nunique()
num_content_duplicates = num_total - num_content_unique

print(f"Tổng số tin nhắn: {num_total}")
print(f"Số lượng nội dung tin nhắn duy nhất: {num_content_unique}")
print(f"Số lượng nội dung tin nhắn bị trùng lặp: {num_content_duplicates}")

Tổng số tin nhắn: 1993
Số lượng nội dung tin nhắn duy nhất: 1993
Số lượng nội dung tin nhắn bị trùng lặp: 0


## Phân tích giá trị khuyết tại cột "Sender"

In [48]:
# Số giá trị khuyết trong cột 'Sender': 109
missing_values = original_data['Sender'].isna().sum()
print(f"Số giá trị khuyết trong cột 'Sender': {missing_values}")

Số giá trị khuyết trong cột 'Sender': 91


In [49]:
# Số giá trị "Sent" trong cột 'Type': 109
received_count = original_data[original_data['Type'] == 'Sent'].shape[0]
print(f"Số giá trị 'Sent' trong cột 'Type': {received_count}")

Số giá trị 'Sent' trong cột 'Type': 91


<p align='justify'>Theo quan sát dữ liệu chúng em rút ra được kết luận rằng cột Sender sẽ bị khuyết nếu đây là tin nhắn do người dùng tự gửi. Cho nên đối với mục đích khi biển đổi dữ liệu của bọn em là chỉ lấy các tin nhắn nhận được thì việc sử dụng cột 'Sent' để thực hiện phân loại sender là hợp lý. </p>

## Lọc Dữ Liệu - Chỉ Giữ Tin Nhắn Nhận Được

<p align='justify'>Dựa trên phân tích ở trên, ta thấy tin nhắn có Type = 'Sent' sẽ có Sender = NaN. Vì mục đích của dự án là phân loại tin nhắn lừa đảo cho tin nhắn nhận được, ta sẽ loại bỏ tất cả các tin nhắn đã gửi.</p>

In [50]:
# Chỉ lấy các dòng có Type = Received
original_data = original_data[original_data['Type'] == 'Received']

## Tạo Dữ Liệu Có Cấu Trúc

Chuyển đổi dữ liệu gốc sang định dạng phù hợp cho việc huấn luyện mô hình:
- Làm sạch nội dung tin nhắn
- Tự động phát hiện URL và số điện thoại
- Phân loại loại người gửi dựa trên số/tên

### Định nghĩa các hàm hỗ trợ

#### Hàm kiểm tra URL trong content
**Lưu ý:** PHẢI kiểm tra thủ công để đảm bảo độ chính xác.

In [51]:
def check_has_url(text):
    if not isinstance(text, str):
        return 0
    
    # 1. Bắt các link có giao thức rõ ràng (http, https, ftp)
    # Ví dụ: http://google.com, https://shb.vn
    protocol_pattern = r'(?:http|ftp|https)://(?:[\w_-]+(?:(?:\.[\w_-]+)+))(?:[\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?'

    # 2. Bắt các link bắt đầu bằng www.
    # Ví dụ: www.vietcombank.com.vn
    www_pattern = r'www\.(?:[\w_-]+(?:(?:\.[\w_-]+)+))(?:[\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?'

    # 3. Bắt các "Naked Domain" (quan trọng nhất cho spam SMS)
    # Logic: Chuỗi ký tự + Dấu chấm + TLD (đuôi tên miền phổ biến)
    # Phải dùng danh sách TLD để tránh bắt nhầm số tiền (ví dụ: 50.000d) hoặc tên người (Mr.Tuan)
    # Danh sách này bao gồm các đuôi phổ biến trong spam SMS Việt Nam
    common_tlds = r'com|vn|net|org|info|xyz|top|club|vip|pro|me|io|edu|gov|biz|cc|tv|site|asia|online|tech'
    
    # Pattern: [tên miền].[tld] (và có thể có /path phía sau)
    # \b đảm bảo không bắt dính vào chữ khác
    naked_domain_pattern = fr'\b[a-zA-Z0-9-]+\.(?:{common_tlds})(?:/[^\s]*)?\b'

    # Kết hợp
    full_pattern = f"{protocol_pattern}|{www_pattern}|{naked_domain_pattern}"

    # Tìm kiếm (IGNORECASE để bắt cả .COM, .VN)
    match = re.search(full_pattern, text, re.IGNORECASE)
    
    return 1 if match else 0

#### Hàm kiểm tra SĐT trong content

**Lưu ý:** Pattern này phát hiện số điện thoại Việt Nam và các định dạng phổ biến. Cần review lại để tránh false positive/negative.

In [52]:
def check_has_phone_number(text):
    if not isinstance(text, str):
        return 0
    
    # Pattern 1: Số di động VN (03x, 05x, 07x, 08x, 09x) hoặc Quốc tế (+84)
    # Cho phép các ký tự ngăn cách như dấu chấm, dấu gạch ngang, khoảng trắng giữa các số
    # Ví dụ bắt được: "0912.345.678", "098 123 4567", "+84912-345-678"
    # Logic: Bắt đầu bằng (+84|84|0), sau đó là 9-10 cụm (ký tự ngăn cách + số)
    mobile_pattern = r'(?:\+84|84|0)(?:[\s.-]*\d){9,10}\b'

    # Pattern 2: Hotline (1800, 1900)
    # Ví dụ bắt được: "1900 1009", "1800.1091"
    hotline_pattern = r'(?:1800|1900)(?:[\s.-]*\d){4,6}\b'

    # Pattern 3: Số điện thoại bàn (02x) - Ít gặp trong spam SMS nhưng vẫn nên có
    landline_pattern = r'(?:02)(?:[\s.-]*\d){9}\b'
    
    # Kết hợp tất cả
    full_pattern = f"{mobile_pattern}|{hotline_pattern}|{landline_pattern}"
    
    # Tìm kiếm (dùng re.IGNORECASE nếu muốn, nhưng số thì không cần)
    match = re.search(full_pattern, text)
    
    return 1 if match else 0

#### Phân loại người gửi

Hàm phân loại dựa trên đặc điểm của sender:
- **Brandname**: Chứa chữ cái (vd: Vietcombank, VinaPhone)
- **Shortcode**: Chỉ số, độ dài 3-8 ký tự (vd: 191, 8088, 19001091)
- **Personal Number**: Số điện thoại cá nhân (bắt đầu 0, 84, +84)
- **Unknown**: Các trường hợp không phù hợp các nhóm trên

In [53]:
def classify_sender(sender):
    # Chuẩn hóa về chuỗi và xóa khoảng trắng thừa (nếu có)
    sender_str = str(sender).strip()

    # 1. BRANDNAME: Chứa ít nhất 1 ký tự chữ cái (A-Z, a-z)
    # Ví dụ: "Vietcombank", "VinaPhone", "CSKH_VIETTEL", "NAPTHE_VT"
    if re.search(r'[a-zA-Z]', sender_str):
        return 'brandname'

    # 2. SHORTCODE (Đầu số ngắn): Chỉ chứa số, độ dài từ 3 đến 8 ký tự
    # Ví dụ: 191, 900, 8088, 19001091 (hotline CSKH)
    # Lưu ý: Hotline 1900xxxx (8 số) thường được xếp vào nhóm dịch vụ/shortcode
    elif re.match(r'^\d{3,8}$', sender_str):
        return 'shortcode'

    # 3. PERSONAL PHONE (SĐT cá nhân):
    # - Bắt đầu bằng 0, 84, hoặc +84
    # - Tổng độ dài phù hợp với SĐT Việt Nam (10 - 12 ký tự số)
    elif re.match(r'^(\+84|84|0)\d{9,10}$', sender_str):
        return 'personal_number'

    # 4. Trường hợp lạ (Số quá dài, ký tự đặc biệt lạ...): Mặc định trả về personal hoặc brandname tùy nhu cầu
    # Thường tin nhắn rác từ nước ngoài hoặc app giả mạo có thể rơi vào đây
    else:
        return 'unknown' # Hoặc gán là 'brandname' nếu muốn gom nhóm

### Tiến hành biển đổi dữ liệu 

In [54]:
# Tạo DataFrame mới với cấu trúc mong muốn
structured_data = pd.DataFrame(columns=['content', 'label', 'has_url', 'has_phone_number', 'sender_type'])

                                    # Cột content:  original_data['Content']
                                    # Xoá kí tự xuống dòng "\n" vì gây khó khăn cho mô hình sau này
structured_data['content'] = original_data['Content'].str.replace('\n', ' ')


                                    # Cột label: 0/1 
structured_data['label'] = None

                                    # Cột has_url: 0/1

structured_data['has_url'] = structured_data['content'].apply(check_has_url)

# Sau đó phải tiến hành kiểm tra thủ công lại

                                    # Cột has_phone_number: 0/1


structured_data['has_phone_number'] = structured_data['content'].apply(check_has_phone_number)

# Sau đó phải tiến hành kiểm tra thủ công lại


                                    # Cột sender_type:
                                    # personal_num/shortcode/brandname

structured_data['sender_type'] = original_data['Sender'].apply(classify_sender)
    

### Lưu kết quả

Dữ liệu đã được cấu trúc và lưu vào file `data/sms_data.csv`.

In [55]:
output_path = 'data/sms_data.csv'
labeled_output_path = 'data/sms_data_for_labeling.csv'
structured_data.to_csv(output_path, index=False)
structured_data.to_csv(labeled_output_path, index=False)

## Kiểm tra sms_data.csv

In [56]:
df = pd.read_csv('data/sms_data.csv')
df.head()


,content,label,has_url,has_phone_number,sender_type
0,[TB] NẠP THẺ ĐỦ ĐẦY - DATA XÀI NGAY! Tặng 20% ...,NaN,1,0,brandname
1,[TB] NÂNG CẤP ĐIỆN THOẠI - NHẬN ƯU ĐÃI 5G. Tặn...,NaN,0,0,brandname
2,[TB] NẠP THẺ ONLINE - DATA XÀI NGAY! Tặng 20% ...,NaN,1,0,brandname
3,[TB] NẠP THẺ ĐỦ ĐẦY - DATA XÀI NGAY! Tặng 20% ...,NaN,1,0,brandname
4,"[TB] LÊN MẠNG, GỌI ĐIỆN THẢ GA! 1. Soạn V90B g...",NaN,0,0,brandname
